### Configuracion inicial

In [ ]:
#Importo librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import ScalarFormatter

In [ ]:
%matplotlib inline
plt.style.use('default')
sns.set()

In [ ]:
#Funciones auxiliares
def mostrar_porcentaje_barplot(ax):
    suma = 0
    for p in ax.patches:
        suma += p.get_height()
    for p in ax.patches:
        ax.annotate(str(np.round(100 *(p.get_height() / suma),decimals=2)) + "%", (p.get_x()+p.get_width()/2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')
def mostrar_valores_barplot(ax):
    for p in ax.patches:
        ax.annotate(np.round(p.get_height(),decimals=2), (p.get_x()+p.get_width()/2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')

In [ ]:
#Cargo el DataFrame
df = pd.read_csv('events.csv', low_memory = False, parse_dates = ['timestamp'], infer_datetime_format = True,
                    dtype = {'event': 'category','condition': 'category','storage': 'category', 'color': 'category', 'staticpage': 'category', 'campaign_source': 'category', 'search_engine': 'category', 'channel': 'category', 'new_vs_returning': 'category', 'region': 'category', 'country': 'category', 'device_type': 'category'})
df.head()

## Chequeos de integridad y calidad de los datos


In [ ]:
df.info()

Como podemos ver, todas las filas especifican 'timestamp', 'event' y 'person'

### Analizamos los valores que pueden tomar las columnas categoricas

In [ ]:
valores = df['condition'].unique()
for valor in valores:
    print(valor, end = ', ')

In [ ]:
valores = df['storage'].unique()
for valor in valores:
    print(valor, end = ', ')

In [ ]:
valores = df['channel'].unique()
for valor in valores:
    print(valor, end = ', ')

In [ ]:
valores = df['staticpage'].unique()
for valor in valores:
    print(valor, end = ', ')

## Analisis tipos de evento

In [ ]:
colUsadasEventos = df.groupby(by = 'event').count()
colUsadasEventos

In [ ]:
columns = list(df)
for event, row in colUsadasEventos.iterrows():
    print(event, end = ':')
    
    for column in columns:
        if(row.get(column) != 0):
            print(' ' + column, end = ',')
            
    print()

## Columnas usadas por cada evento

Todos los eventos contienen informacion sobre **'timestamp'**, **'event'** y **'person'**, y ademas utilizan las siguientes columnas:

**ad campaign hit:**
    url, campaign_source

**brand listing:**
    skus

**checkout:** 
    sku, model, condition, storage, color

**conversion:**
    sku, model, condition, storage, color

**generic listing:**
    skus

**lead:**
    model

**search engine hit:** 
    search_engine

**searched products:** 
    skus, search_term

**staticpage:** 
    staticpage

**viewed product:** 
    sku, model, condition, storage, color

**visited site:** 
    channel, new_vs_returning, city, region, country, device_type, screen_resolution, operating_system_version, browser_version


### Captacion de usuarios

In [ ]:
personas_new_ret = df.loc[df["event"] == "visited site", ['person', 'new_vs_returning']]
personas_new_ret['returning_hits'] = personas_new_ret['new_vs_returning'] == 'Returning'
personas_new_ret.drop(columns='new_vs_returning', inplace=True)
personas_new_ret.head()

In [ ]:
personas_ret_hits = personas_new_ret.groupby(by = 'person').agg({'returning_hits': np.count_nonzero})
personas_ret_hits.head(10)

In [ ]:
x = personas_ret_hits['returning_hits']
x.value_counts().head(10)

In [ ]:
ax = x.hist(bins = 50)
ax.set_yscale('log')
ax.set_title('Cantidad de usuarios según returning_hits')
ax.set_xlabel('Cantidad de returning_hits')

La mayoría de los usuarios ingresa al sitio una única vez

In [ ]:
cant_returns = x.value_counts()
cero_returns = cant_returns[0]
uno_diez_returns = cant_returns[1:11].values.sum()
diez_mas_returns = cant_returns[11:].values.sum()
datos_returns = [cero_returns, uno_diez_returns, diez_mas_returns]
x_labels = ['1', '2 - 10', '10+']
ax = sns.barplot(x = x_labels, y = datos_returns)
ax.set(xlabel='Cantidad de visitas al sitio', ylabel='Cantidad de Usuarios', title = 'Cantidad de usuarios segun veces que visitaron el sitio')

La mayoria de los usuarios visita el sitio una única vez, mientras que solo una pequeña proporción volvió al sitio mas de 10 veces. Es posible que un mismo usuario entre desde dispositivos diferentes.

## Analisis temporal

In [ ]:
#Analizamos periodo de los datos
df['timestamp'].dt.year.value_counts()

Todos los datos son de 2018

In [ ]:
df['timestamp'].dt.month_name().value_counts()

De los primeros dos trimestres del año

In [ ]:
#Vamos a hacer un analisis sobre los dias en los que se produjeron los eventos
dias_semana = df['timestamp'].dt.weekday_name
dias_ordenados = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dias_semana = pd.Categorical(dias_semana, categories = dias_ordenados, ordered = True)
g = dias_semana.value_counts().plot('bar', figsize = (9, 5))
g.set_title('Cantidad de eventos por dia de la semana')
g.set_xticklabels(['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo'])
plt.xticks(rotation = 'horizontal')

Vemos que durante el fin de semana el trafico es menor

In [ ]:
tiempo_eventos = pd.DataFrame()
tiempo_eventos['event'] = df['event']
tiempo_eventos['dia_semana'] = df['timestamp'].dt.weekday_name
tiempo_eventos['dia'] = df['timestamp'].dt.day
tiempo_eventos['hora'] = df['timestamp'].dt.hour
tiempo_eventos['mes'] = df['timestamp'].dt.month_name()

#hora_weekday_organic = tiempo_eventos[tiempo_eventos['channel'] == 'organic'].groupby('dia_semana')['hora'].value_counts().to_frame()
#hora_weekday_paid = tiempo_eventos[tiempo_eventos['channel'] == 'paid'].groupby('dia_semana')['hora'].value_counts().to_frame()

hora_weekday_visitas = tiempo_eventos[tiempo_eventos['event'] == 'visited site'].groupby('dia_semana')['hora'].value_counts().to_frame()
hora_weekday_visitas = hora_weekday_visitas.rename(columns = {'hora':'cant_casos'})
hora_weekday_visitas = hora_weekday_visitas.reset_index('hora')
hora_weekday_visitas.index = pd.CategoricalIndex(hora_weekday_visitas.index, categories= dias_ordenados)
hora_weekday_visitas.sort_index(level=0, inplace=True)
hora_weekday_visitas = hora_weekday_visitas.reset_index()

dia_mes_visitas = tiempo_eventos[tiempo_eventos['event'] == 'visited site'].groupby('mes')['dia'].value_counts().to_frame()
dia_mes_visitas = dia_mes_visitas.rename(columns = {'dia':'cant_casos'})
dia_mes_visitas = dia_mes_visitas.reset_index('dia')
dia_mes_visitas.index = pd.CategoricalIndex(dia_mes_visitas.index, categories= ['January', 'February', 'March', 'April', 'May', 'June'])
dia_mes_visitas.sort_index(level=0, inplace=True)
dia_mes_visitas = dia_mes_visitas.reset_index()

In [ ]:
pivot = hora_weekday_visitas.pivot_table(index = 'dia_semana', columns = 'hora', values = 'cant_casos')
cmap = sns.cm.rocket_r
sns.set(rc={'figure.figsize':(13,4)})
ax = sns.heatmap(data = pivot, cmap = cmap)
ax.set(xlabel='Hora', ylabel='', title = 'Visitas segun hora del dia')
ax.set_yticklabels(['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo'])

Durante el fin de semana la cantidad de visitas es menor, mientras que la mayoria de las visitas se realizan entre las 12 y las 2

In [ ]:
pivot = dia_mes_visitas.pivot_table(index = 'dia', columns = 'mes', values = 'cant_casos')
cmap = sns.cm.rocket_r
sns.set(rc={'figure.figsize':(12,12)})
ax = sns.heatmap(data = pivot, cmap = cmap, annot = True, fmt=".0f")
ax.set(xlabel='Mes', ylabel='Dia', title = 'Visitas 2018')
ax.set_xticklabels(['Enero','Febrero','Marzo','Abril','Mayo','Junio'])

Las visitas crecieron aceleradamente a lo largo del año, logrando un despegue a partir de mayo. Analizaremos las causas de esto mas adelante

In [ ]:
usuarios_semanal = df.loc[df['event'] == 'visited site', ['timestamp', 'person']].copy()
usuarios_semanal['semana'] = usuarios_semanal['timestamp'].dt.week - 1
usuarios_semanal = usuarios_semanal.drop_duplicates(subset = ['semana', 'person'])
usuarios_semanal = usuarios_semanal[usuarios_semanal['semana'] != usuarios_semanal['semana'].max()] #Borro la ultima semana incompleta
usuarios_semanal['semana'] = pd.to_datetime('2018', format='%Y') + pd.to_timedelta(usuarios_semanal['semana'].mul(7).astype('str') + ' days')

semanas = usuarios_semanal['semana'].value_counts().to_frame().sort_index()
ax = semanas['semana'].plot()
ax.set(xlabel='Semana', ylabel='Usuarios unicos')
ax.set_title('Usuarios unicos semanales')

In [ ]:
usuarios_semanal = df.loc[df['new_vs_returning'] == 'Returning', ['timestamp']].copy()
usuarios_semanal['semana'] = usuarios_semanal['timestamp'].dt.week - 1
usuarios_semanal = usuarios_semanal[usuarios_semanal['semana'] != usuarios_semanal['semana'].max()] #Borro la ultima semana incompleta
usuarios_semanal['semana'] = pd.to_datetime('2018', format='%Y') + pd.to_timedelta(usuarios_semanal['semana'].mul(7).astype('str') + ' days')

semanas = usuarios_semanal['semana'].value_counts().to_frame().sort_index()
ax = semanas['semana'].plot()
ax.set(xlabel='Semana', ylabel='Usuarios nuevos')
ax.set_title('Usuarios nuevos semanales')

La cantidad de usuarios nuevos que visitan el sitio disminuye a partir de la segunda semana de mayo

## Origen de las visitas

In [ ]:
visitas = df.loc[df["event"] == "visited site", ['timestamp', 'person', 'channel', 'new_vs_returning', 'city', 'region', 'country', 'device_type', 'screen_resolution', 'operating_system_version', 'browser_version']]
colors = {'Paid': 'navy', 'Direct': 'g', 'Organic': 'coral', 'Referral': 'm', 'Social': 'goldenrod', 'Email': 'k', 'Unknown': 'grey'}

In [ ]:
valores = visitas['channel'].value_counts()
ax = valores.plot(kind = 'bar', color=[colors.get(i) for i in valores.index])
ax.set_title('Origen de las visitas')
ax.set_ylabel('Cantidad')
ax.set_xlabel('Channel')
plt.xticks(rotation=0)
mostrar_porcentaje_barplot(ax)

### Origen de usuarios nuevos

In [ ]:
valores = visitas[visitas['new_vs_returning'] == 'New']['channel'].value_counts()
ax = valores.plot(kind = 'bar', color=[colors.get(i) for i in valores.index])
ax.set_title('Origen de las visitas de usuarios nuevos')
ax.set_ylabel('Cantidad')
ax.set_xlabel('Channel')
plt.xticks(rotation=0)
mostrar_porcentaje_barplot(ax)

#### Evolucion canales a lo largo del tiempo para usuarios nuevos

In [ ]:
evolucion_semanal = visitas.copy()
evolucion_semanal['mes'] = evolucion_semanal['timestamp'].dt.month
evolucion_semanal = evolucion_semanal[evolucion_semanal['new_vs_returning'] == 'New']

evolucion_semanal = pd.crosstab(evolucion_semanal['mes'], evolucion_semanal['channel'])
evolucion_semanal = evolucion_semanal.div(evolucion_semanal.sum(axis=1), axis=0).multiply(100)

ax = evolucion_semanal.plot(kind = 'area', figsize = (10,7))
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
ax.set(xlabel='Mes', ylabel='Porcentaje', title = 'Proporcion de visitas usuarios nuevos por channel')

Vemos que los porcentajes se mantienen mas o menos constantes a lo largo del tiempo

### Origen usuarios no nuevos

In [ ]:
valores = visitas[visitas['new_vs_returning'] == 'Returning']['channel'].value_counts()
ax = valores.plot(kind = 'bar', color=[colors.get(i) for i in valores.index])
ax.set_title('Origen de las visitas de usuarios no nuevos')
ax.set_ylabel('Cantidad')
ax.set_xlabel('Channel')
plt.xticks(rotation=0)
mostrar_porcentaje_barplot(ax)

Vemos que el canal pago es el que mas visitas atrae, no solo para usuarios nuevos sino tambien para usuarios que ya habian entrado al sitio. Por otro lado, se puede ver como el canal "Direct" (tipeado de URL, acceso mediante favoritos y similares) toma protagonismo para visitas de usuarios que ya conocen el sitio. Pero esto puede ser causado por un pequeño numero de usuarios que visitan repetidas veces el sitio

#### Evolución de los canales para visitas de usuarios no nuevos

In [ ]:
evolucion_semanal = visitas
evolucion_semanal['mes'] = evolucion_semanal['timestamp'].dt.month
evolucion_semanal = evolucion_semanal[evolucion_semanal['new_vs_returning'] == 'Returning']

evolucion_semanal = pd.crosstab(evolucion_semanal['mes'], evolucion_semanal['channel'])
evolucion_semanal = evolucion_semanal.div(evolucion_semanal.sum(axis=1), axis=0).multiply(100)

sns.set(rc={'figure.figsize':(12,12)})
ax = evolucion_semanal.plot(kind = 'area')
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
ax.set(xlabel='Mes', ylabel='Porcentaje', title = 'Proporcion de visitas usuarios no nuevos por channel')

Vemos un aumento en proporcion del canal pago y una disminucion del canal directo a lo largo del tiempo.

Ahora tomaremos solo un metodo predominante de acceso al sitio por usuario, para usarios no nuevos

In [ ]:
valores = visitas[visitas['new_vs_returning'] == 'Returning'].groupby('person')['channel'].value_counts()
valores = valores.reset_index(level='channel', name='Counts')
valores = valores[~valores.index.duplicated(keep='first')]['channel'].value_counts()
                  
ax = valores.plot(kind = 'bar', color=[colors.get(i) for i in valores.index])
ax.set_title('Canal predominante de ingreso al sitio para usuarios no nuevos')
ax.set_ylabel('Cantidad de usuarios')
ax.set_xlabel('Evento predominante')
plt.xticks(rotation=0)
mostrar_porcentaje_barplot(ax)

Vemos que en este caso el canal predominante de ingreso al sitio para usuarios no nuevos sigue siendo el canal pago, aunque con menor proporcion que para usuarios nuevos. El ingreso mediante "referrals" de otros sitios queda en segundo lugar, teniendo una proporcion mucho mayor que para usuarios nuevos.

## Los usuarios que hicieron checkout. ¿Concretaron la conversión?

Obtenemos la información del set de datos

In [ ]:
checkouts = df.loc[df['event'] == 'checkout',["sku","timestamp","person","model","condition","storage","color"]]
conversiones = df.loc[df['event'] == 'conversion',["sku","timestamp","person","model","condition","storage","color"]]

### Observemos la evolución de los checkouts en el tiempo

In [ ]:
checkouts_by_week = checkouts.loc[:, ['timestamp']]
checkouts_by_week['semana'] = checkouts_by_week['timestamp'].dt.week - 1
checkouts_by_week = checkouts_by_week[checkouts_by_week['semana'] != checkouts_by_week['semana'].max()] #Borro la ultima semana incompleta
checkouts_by_week['semana'] = pd.to_datetime('2018', format='%Y') + pd.to_timedelta(checkouts_by_week['semana'].mul(7).astype('str') + ' days')

checkouts_weeks = checkouts_by_week['semana'].value_counts().to_frame().sort_index()
ax = checkouts_weeks.plot(figsize=(8,5))
ax.set(xlabel='Semana', ylabel='Cantidad de checkouts')
ax.set_title('Checkouts semanales')

Aparentemente, algo ocurrió a mitad de Mayo. Haciendo zoom podemos ver que el 14 comenzaron a aumentar los checkouts.

In [ ]:
acotado = checkouts.loc[(checkouts['timestamp'] > pd.to_datetime("2018-05-11")) & (checkouts['timestamp'] < pd.to_datetime("2018-05-18"))].copy()
acotado['fecha'] = acotado['timestamp'].dt.date

ax = acotado['fecha'].value_counts().plot(marker=".",markersize="20",figsize=(8,5))
ax.set(xlabel='Dia', ylabel='Cantidad de checkouts')
ax.set_title('Checkouts 11/05 al 17/05')

Ahora analicemos la evolución de las conversiones.

In [ ]:
conversiones_by_week = conversiones.loc[:, ["timestamp"]]
conversiones_by_week['semana'] = conversiones_by_week['timestamp'].dt.week - 1
conversiones_by_week = conversiones_by_week[conversiones_by_week['semana'] != conversiones_by_week['semana'].max()] #Borro la ultima semana incompleta
conversiones_by_week['semana'] = pd.to_datetime('2018', format='%Y') + pd.to_timedelta(conversiones_by_week['semana'].mul(7).astype('str') + ' days')

conversiones_weeks = conversiones_by_week['semana'].value_counts().to_frame().sort_index()
ax = conversiones_weeks.plot(figsize=(8,5))
ax.set(xlabel='Semana', ylabel='Cantidad de conversiones')
ax.set_title('Conversiones semanales')

Veamos si hay correlación entre el aumento de checkouts y las conversiones.

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:green'
ax1.set_xlabel('Semana')
ax1.set_ylabel('Cantidad de checkouts', color=color)
ax1.plot(checkouts_weeks['semana'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()

color = 'tab:blue'
ax2.set_ylabel('Cantidad de conversiones', color=color)
ax2.plot(conversiones_weeks['semana'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.show()


Se observa un aumento significativo tanto de conversiones como de checkouts en la segunda semana de mayo, pero de ahi en adelante solo los checkouts siguieron aumentando mientras que las conversiones disminuyeron. Analizaremos la proporcion de conversiones sobre checkouts hasta esa semana.

In [ ]:
checkouts_hasta_mayo = checkouts_weeks.iloc[0:18]
conversiones_hasta_mayo = conversiones_weeks.iloc[0:18]

fig, ax1 = plt.subplots(figsize=(10,5))

color = 'tab:green'
ax1.set_xlabel('Semana')
ax1.set_ylabel('Cantidad de checkouts', color=color)
ax1.plot(checkouts_hasta_mayo, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()

color = 'tab:blue'
ax2.set_ylabel('Cantidad de conversiones', color=color)
ax2.plot(conversiones_hasta_mayo, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.show()

In [ ]:
porcentaje_conversiones_sobre_checkouts = (conversiones_weeks['semana'] / checkouts_weeks['semana']) * 100
ax = porcentaje_conversiones_sobre_checkouts.plot(title="Porcentaje de conversiones sobre checkouts")
ax.set_ylabel("Porcentaje")
ax.set_xlabel("Fecha")

Vemos que hasta mayo la cantidad de checkouts y conversiones estaban intimamente relacionadas, representando alrededor de un 15% la cantidad de conversiones sobre la cantidad de checkouts. Debemos analizar que sucede en mayo que hace que de ahi en adelante suban los checkouts pero bajen las conversiones.

### Marcas con mas checkouts y conversiones

In [ ]:
def obtener_marca(serie_de_modelo):
    # La marca es siempre la primer palabra del modelo
    r = []
    for modelo in serie_de_modelo.tolist():
        if pd.isna(modelo):
            r.append("Unknown")
        else:
            r.append(modelo.split()[0])
    return r

Analicemos los checkouts de cada marca

In [ ]:
checkouts["Marca"] = obtener_marca(checkouts["model"])
checkouts = checkouts.loc[checkouts["Marca"] != "Unknown"] # Remuevo datos erroneos
checkouts["Checkouts"] = 1
checkouts_by_marca = checkouts[["Marca","Checkouts"]].groupby(by=["Marca"]).agg("count")\
    .sort_values(by="Checkouts",ascending=False).iloc[::-1]
p = checkouts_by_marca.plot(kind="barh",title="Checkouts de cada marca",legend=False)
p.set_ylabel("Marca")
p.set_xlabel("Checkouts")

### ¿Las conversiones mantienen la relación?

In [ ]:
conversiones["Marca"] = obtener_marca(conversiones["model"])
conversiones = conversiones.loc[conversiones["Marca"] != "Unknown"] # Remuevo datos erroneos
conversiones["Conversiones"] = 1
conversiones_by_marca = conversiones[["Marca","Conversiones"]].groupby(by=["Marca"]).agg("count")\
    .sort_values(by="Conversiones",ascending=False).iloc[::-1]
p = conversiones_by_marca.plot(kind="barh",figsize=(9,7),fontsize=15,title="Conversiones de cada marca")
p.set_xlabel("Conversiones")
p.legend(loc="lower right")
p

Es mejor analizar esta información de chekouts y conversiones en un mismo grafico, para poder apreciar mejor los datos.

In [ ]:
marca_stats = conversiones_by_marca.join(checkouts_by_marca).sort_values(by="Checkouts",ascending=False)
marca_stats["No concretados"] = marca_stats["Checkouts"] - marca_stats["Conversiones"]
marca_stats = marca_stats.rename(columns={'Conversiones': 'Concretados'}).iloc[::-1]
marca_stats.drop("Checkouts", axis=1, inplace= True)
p = marca_stats.plot(kind="barh",stacked=True,figsize=(9,7),fontsize=15,\
                 title="Checkouts de cada marca")
p.legend(loc="lower right")
p.set_xlabel("Checkouts")
p

## ¿Cuál es la probabilidad de concretar un checkout? 

A continuación analizaremos la probabilidad que posee cada marca. Para tener un resultado fiel, filtramos aquellas marcas que tienen menos de 1000 checkouts.

In [ ]:
t = checkouts_by_marca.join(conversiones_by_marca)
t = t[t["Checkouts"] > 1000]
t["Porcentaje"] = (t["Conversiones"] *100)/t["Checkouts"]
t["Porcentaje"] = t["Porcentaje"].fillna(0)
t = t.drop(columns=["Conversiones","Checkouts"])
p = t.sort_values("Porcentaje")\
    .plot(kind="barh",legend=False,title="Probabilidad de concretar checkouts por marca",figsize=(8,2))
p.set_xlabel("Porcentaje")


In [ ]:
view_products = df.loc[df["event"] == "viewed product",["model"]].copy()
view_products["Marca"] = obtener_marca(view_products["model"])
view_products["Visitas"] = 1

In [ ]:
view_products_by_marca = view_products.drop(columns=["model"]).groupby(by="Marca").agg("sum")

p = view_products_by_marca.sort_values(by="Visitas",ascending=False)\
    .iloc[::-1].plot(kind="barh",title="Marcas más visitadas",figsize=(10,7),fontsize=14,legend=False)
p.set_xlabel("Visitas")
p

In [ ]:
marca_stats = view_products_by_marca.join(checkouts_by_marca,on="Marca",how="outer")\
    .join(conversiones_by_marca,on="Marca",how="outer")
marca_stats['Visitas']= (marca_stats['Visitas']/marca_stats['Visitas'].sum()) *100
marca_stats['Checkouts']= (marca_stats['Checkouts']/marca_stats['Checkouts'].sum()) *100
marca_stats['Conversiones']= (marca_stats['Conversiones']/marca_stats['Conversiones'].sum()) * 100
marca_stats = marca_stats.fillna(0)
marca_stats = marca_stats.T

Para el siguiente gráfico, hemos decidido utilizar una librería externa. 


Se puede instalar corriendo: pip install plotly

O en Anaconda: conda install -c plotly plotly


In [ ]:
import plotly
plotly.offline.init_notebook_mode(connected=True)
import plotly.offline as py
import plotly.graph_objs as go

data = [go.Scatterpolar(
  r = marca_stats["Samsung"].tolist(),
  theta = ['Visitas','Checkouts','Conversiones'],
    name = 'Samsung',
  fill = 'toself'
),go.Scatterpolar(
  r = marca_stats["iPhone"].tolist(),
  theta = ['Visitas','Checkouts','Conversiones'],
    name = 'iPhone',
  fill = 'toself'
),go.Scatterpolar(
  r = marca_stats["Motorola"].tolist(),
  theta = ['Visitas','Checkouts','Conversiones'],
    name = 'Motorola',
    fill = 'toself'
)]

layout = go.Layout(
  polar = dict(
    radialaxis = dict(
      visible = True,
      range = [0, 60]
    )
  ),
    title='Porcentaje de visitas, checkouts, y conversiones de marcas',
    font= dict(
        size =18
    ),
  showlegend = True
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

## Observemos la información que nos dan los modelos principales

In [ ]:
p = view_products.groupby(by=["model"]).agg("sum").sort_values(by="Visitas",ascending=False).head(20)\
    .iloc[::-1].plot(kind="barh",title="20 celulares con más visitas",figsize=(10,7),fontsize=14,legend=False)
p.set_ylabel("Modelo")
p.set_xlabel("Visitas")
p

Los gráficos anteriores pueden agruparce para apreciar mejor la informacion.

In [ ]:
view_products = df.loc[(df["event"] == "viewed product")|(df["event"] == "checkout")|(df["event"] == "conversion"),["model",'event']].copy()
view_products["Marca"] = obtener_marca(view_products["model"])
view_products["Visitas"] = view_products['event']=='viewed product'
view_products['checkouts']= view_products['event']=='checkout'
view_products['compras']= view_products['event']=='conversion'    
view_products_by_model= view_products.groupby(by=["model"]).agg({'Visitas':'sum','checkouts':'sum','compras':'sum'}).sort_values(by="Visitas",ascending=False).head(20)


fig, (ax, ax2, ax3) = plt.subplots(ncols=3, sharey=True)


ax.yaxis.tick_left()

view_products_by_model['Visitas'].iloc[::-1].plot(kind="barh",title="20 celulares con más visitas",figsize=(10,7),fontsize=10,legend=False,ax=ax)
view_products_by_model['checkouts'].iloc[::-1].plot(kind="barh",title="checkouts",figsize=(10,7),fontsize=10,legend=False,ax=ax2)
view_products_by_model['compras'].iloc[::-1].plot(kind="barh",title="compras",figsize=(10,7),fontsize=10,legend=False,ax=ax3)

plt.show()

#### Analsis de checkouts por persona

Veo si existen casos de double tracking

In [ ]:
def min_diferencia_tiempo(df):
        min_diferencia=pd.to_timedelta('365 days')
        for i in range(1,len(df)):
            dif = df.iloc[i]-df.iloc[i-1]
            if dif<min_diferencia:
                min_diferencia = dif
        return min_diferencia    
            
checkouts_por_persona = df.loc[df['event']=='checkout',['timestamp','person','model', 'sku']]
duplicados = checkouts_por_persona[checkouts_por_persona.duplicated(subset=['person'], keep=False)]
min_diferencia_checkouts = duplicados.groupby('person').agg({'timestamp':min_diferencia_tiempo}).sort_values(by='timestamp')
min_diferencia_checkouts['menos_de_1seg'] = min_diferencia_checkouts['timestamp']<pd.to_timedelta('00:00:01')
min_diferencia_checkouts['menos_de_1seg'].value_counts()

Existe solo 1 caso de todos los checkouts que podría ser de double tracking, lo examino

In [ ]:
min_diferencia_checkouts.loc[min_diferencia_checkouts['menos_de_1seg']==True]

In [ ]:
checkouts_por_persona.loc[checkouts_por_persona['person']=='0ee73fbe']

Los dos eventos de checkout que son en el mismo momento son de modelos distintos, por lo tanto supondremos que no es un caso de double tracking

### Analizo la cantidad de checkouts por persona 

In [ ]:
checkouts_por_persona['checkouts'] = 1
checkouts_por_persona["marca"] = obtener_marca(checkouts_por_persona["model"])
for marca in ['Samsung','iPhone','Motorola','LG','Lenovo','Sony','Quantum','iPad','Asus']:
    checkouts_por_persona[marca] = checkouts_por_persona['marca'] == marca

    

In [ ]:

cantidad_de_checkouts_por_persona = checkouts_por_persona.groupby('person').agg({'checkouts':sum,\
        'Samsung':sum,'iPhone':sum,'Motorola':sum,'LG':sum,'Lenovo':sum,'Sony':sum,\
        'Quantum':sum,'iPad':sum,'Asus':sum})
g = cantidad_de_checkouts_por_persona['checkouts'].value_counts().plot('bar')
g.set_title('Cantidad de checkouts por persona')
g.set_xlabel('Cantidad de checkouts')
g.set_ylabel('Numero de personas')

In [ ]:
cant_checkouts_log=cantidad_de_checkouts_por_persona['checkouts'].value_counts()#.transform(lambda x: np.log(x)+1 )
g_log = cant_checkouts_log.plot('bar')
g_log.set_yscale('log')
#g_log.yaxis.set_major_formatter(ScalarFormatter())
g_log.set_title('Cantidad de checkouts por persona [Log]')
g_log.set_xlabel('Cantidad de checkouts')
g_log.set_ylabel('Cantidad de personas')
mostrar_valores_barplot(g_log)

#### Verifico si los distintos checkouts son sobre los mismos modelos

In [ ]:
a = checkouts_por_persona.groupby('person').agg({'model':'nunique','checkouts':'count'})
g = a.groupby('checkouts').agg({'model':'mean'}).plot(kind='bar',alpha=0.5, legend=False)
a.loc[a['checkouts']==24]
g.set_title('Promedio de modelos dintintos por cantidad de checkouts')
g.set_xlabel('Número de checkouts')
g.set_ylabel('Número de modelos distintos vistos')

#### Hago un analisis de las marcas que mas se aparecen en funcion de la cantidad de checkouts que hizo cada cliente

In [ ]:
multiples = cantidad_de_checkouts_por_persona.loc[cantidad_de_checkouts_por_persona['checkouts']>1]

my_colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#bcbd22", "#17becf"]  
g = multiples.groupby('checkouts').agg({'Samsung':sum,'iPhone':sum,'Motorola':sum,'LG':sum,'Lenovo':sum,\
        'Sony':sum,'Quantum':sum,'iPad':sum,'Asus':sum}).iloc[:,:5].plot(kind='line',color = my_colors)
g.set_title('Distribucion de las marcas en funcion de la cantidad de checkouts por persona')
g.set_xlabel('Cantidad de checkouts')
g.set_ylabel('Cantidad de apariciones')


#### Hago un analisis de los productos visitados


In [ ]:
productos_visitados = df.loc[df['event']=='viewed product',['timestamp','person','sku','model','condition']]


In [ ]:
productos_visitados = df.loc[df['event']=='viewed product',['person','sku','model','condition']]
productos_visitados['visita'] = 1
cant_productos_visitados = productos_visitados.groupby('person').agg({'visita':'count'})['visita'].value_counts()


In [ ]:
cant_productos_visitados = productos_visitados.groupby('person').agg({'visita':'count'})
g = cant_productos_visitados ['visita']\
    .transform(lambda x: np.log(x)+1 ).plot('hist',alpha=0.75, bins= 20)
g.set_title('Histograma de Viewed Product | Productos visitados [Log] ')
g.set_xlabel('Cantidad de productos visitados')

In [ ]:
merged = pd.merge(cant_productos_visitados, cantidad_de_checkouts_por_persona, on='person', how='left')
# Estamos sacando aquellas personas que fueron directo al checkout (mediante publicidad) sin observar productos

g = merged[["visita","checkouts"]].transform(lambda x: np.log(x)+1 ).plot(kind='hist',alpha=0.80,bins = 10, color =["#ff7f0e","#2ca02c"])
#merged[["checkouts","visita"]].plot(kind='bar', stacked=True)
g.set_title('Frecuencia de Checkouts x Producto Visitado [Log]')

## Leads

In [ ]:
leads = df.loc[df['event'] == 'lead', ['timestamp', 'person', 'model']].copy()
conversions = df.loc[df['event'] == 'conversion', ['timestamp', 'person', 'model']]
checkouts = df.loc[df['event'] == 'checkout', ['timestamp', 'person', 'model']]

### Cantidad de leads

In [ ]:
len(leads['timestamp'])

Hubo 380 solicitudes de notificacion de stock

### Cantidad de usuarios que solicitaron notificacion de stock

In [ ]:
len(leads['person'].unique())

291 usuarios solicitaron notificacion por faltante de stock

In [ ]:
len(conversions['person'].unique())

716 usuarios realizaron al menos una conversion

### Leads semanales

In [ ]:
leads_by_week = leads.loc[:, ['timestamp']].copy()
leads_by_week['semana'] = leads_by_week['timestamp'].dt.week - 1
leads_by_week = leads_by_week[leads_by_week['semana'] != leads_by_week['semana'].max()] #Borro la ultima semana incompleta
leads_by_week['semana'] = pd.to_datetime('2018', format='%Y') + pd.to_timedelta(leads_by_week['semana'].mul(7).astype('str') + ' days')

leads_weeks = leads_by_week['semana'].value_counts().to_frame().sort_index()
ax = leads_weeks['semana'].plot()
ax.set(xlabel='Semana', ylabel='Solicitudes de stock')
ax.set_title('Solicitudes de stock semanales')

In [ ]:
leads_weeks['semana'].iloc[-11:].mean()

Hay un promedio de 30 leads semanales desde abril.

### Relacion leads y conversiones

In [ ]:
conversions_by_week = conversions.loc[:, ['timestamp']].copy()
conversions_by_week['semana'] = conversions_by_week['timestamp'].dt.week - 1
conversions_by_week = conversions_by_week[conversions_by_week['semana'] != conversions_by_week['semana'].max()] #Borro la ultima semana incompleta
conversions_by_week['semana'] = pd.to_datetime('2018', format='%Y') + pd.to_timedelta(conversions_by_week['semana'].mul(7).astype('str') + ' days')
conversions_weeks = conversions_by_week['semana'].value_counts().to_frame().sort_index()

fig, ax1 = plt.subplots()

color='#B40404'
ax1.set_xlabel('Semana')
ax1.set_ylabel('Solicitudes de stock', color=color)
ax1.plot(leads_weeks['semana'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()

color = 'tab:blue'
ax2.set_ylabel('Compras', color=color)
ax2.plot(conversions_weeks['semana'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
ax1.set_title('Solicitudes de stock y compras semanales')
plt.show()

Notamos que en los momentos en que mas solicitudes de stock hay, las conversiones se estancan o disminuyen

### ¿Que marcas tienen mas solicitudes de notificacion de stock?

In [ ]:
def obtener_marca(serie_de_modelo):
    # La marca es siempre la primer palabra del modelo
    r = []
    for modelo in serie_de_modelo.tolist():
        if pd.isna(modelo):
            r.append("Unknown")
        else:
            r.append(modelo.split()[0])
    return r

In [ ]:
leads['marca'] = obtener_marca(leads['model'])
leads['semana'] = leads['timestamp'].dt.week
conversions['marca'] = obtener_marca(conversions['model'])

leads_unicos = leads.drop_duplicates(subset = ['person', 'model', 'semana'])

In [ ]:
leads_count = leads_unicos['marca'].value_counts().to_frame()
conversions_count = conversions['marca'].value_counts().to_frame()
count = pd.concat([leads_count, conversions_count], axis=1, join_axes=[leads_count.index])
count.columns = ['leads','conversiones']
count = count.iloc[0:5]

fig, (ax, ax2) = plt.subplots(ncols=2, sharey=True)

ax.yaxis.tick_left()

count['leads'].plot(kind='barh', x='LABEL',  legend=False, ax=ax)
count['conversiones'].plot(kind='barh', x='LABEL',ax=ax2)
ax.set_title('Solicitudes de stock')
ax.set(xlabel='Solicitudes de stock totales', ylabel='Marca')
ax2.set_title('Compras')
ax2.set(xlabel='Compras totales')
plt.show()

Vemos que los iphone son los que mayor cantidad de leads tienen y los Samsung quedan en segundo lugar, pero la cantidad de Samsung vendidos es el doble que la cantidad de iphones vendidos, lo que significa que hay un problema de stock para los iphone. Tambien para los LG hay una cantidad grande de lead en comparacion con sus conversiones.

### ¿Cuantos de esos lead se convirtieron en compra?

In [ ]:
leads_red = leads_unicos[['person', 'model']]
conversions_red = conversions[['person', 'model']].copy()
conversions_red['conversion'] = True

leads_red = leads_red.merge(conversions_red.drop_duplicates(), on = ['person', 'model'], how = 'left')
valores = leads_red['conversion'].fillna(False).value_counts()
valores.rename({False:'No compro', True:'Compro'}, inplace=True)
ax = valores.plot('pie', autopct='%1.0f%%', figsize=(4,4), title='Cantidad de solicitudes que se convirtieron en compra')
ax.set_ylabel('')

Solamente 29 de los 380 leads unicos (sin contar multiples lead de un mismo usuario para un mismo modelo en una misma semana) se convirtieron en compra, un numero desalentador considerando que son 351 potenciales compras que no pudieron realizarse por faltante de stock.

In [ ]:
len(conversions['timestamp'])

In [ ]:
comprados_leads = pd.Series({'Compras':len(conversions['timestamp']), 'Solicitudes de stock sin compra':380 - 29})
ax = comprados_leads.plot('bar')
plt.xticks(rotation=0)
ax.set(ylabel='Cantidad total')
ax.set_title('Compras totales vs solicitudes de stock totales')

In [ ]:
100 * (1 - (1172/(1172+351)))

Se vendió un 23% menos cantidad de celulares de lo que se podría haber vendido si hubiese habido stock suficiente.

# Analisis de la condición de los celulares

#### Mostramos el evento según la condicion que se encuentra el celular

In [ ]:
aux=df.loc[((df['event'] == 'viewed product') | (df['event'] == 'checkout') | (df['event'] == 'conversion')), ['event','condition']]
condicion_evento = pd.crosstab(aux['condition'], aux['event'])
condicion_evento

#### Hacemos el gráfico del mismo

In [ ]:
condicion_evento_norm = pd.DataFrame()
condicion_evento_norm['viewed product'] = (condicion_evento['viewed product'] / condicion_evento['viewed product'].sum()) * 100
condicion_evento_norm['checkout'] = (condicion_evento['checkout'] / condicion_evento['checkout'].sum()) * 100
condicion_evento_norm['conversion'] = (condicion_evento['conversion'] / condicion_evento['conversion'].sum()) * 100
sns.set(rc={'figure.figsize':(8.7,8.27)})
ax = condicion_evento_norm.T.plot.bar(stacked=True)
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
ax.set(xlabel='Evento', ylabel='Porcentaje', title = 'Proporcion de condicion de celular por evento')

Como se puede ver en el gráfico, la gente tiende a buscar y comprar más los celulares que no estan en las mejores condiciones (bom). La participacion de los celulares nuevos es casi inexistente. Vemos un decremento en la participacion de celulares de condicion excelente en las conversiones.

In [ ]:
g = sns.countplot(x="condition", hue="event", data=aux, palette="hls")
g.set_title("Cantidad de eventos por condicion del celular", fontsize=18)
g.set_xlabel("condicion", fontsize=18)
g.set_ylabel("Cantidad de eventos", fontsize=18)

no se puede ver bien porque la cantidad de viewed product

In [ ]:
auxiliar_celulares2 =df.loc[(pd.notnull(df['sku'])),['sku','condition']].drop_duplicates()
auxiliar_celulares2.reset_index(drop='true')
auxiliar_celulares2.head(5)

In [ ]:
auxiliar_celulares2['celulares'] = 1
cantidad_de_celulares = auxiliar_celulares2.groupby('condition').agg({'celulares':'sum'})

In [ ]:
cantidad_de_celulares

In [ ]:
df1= df.loc[(df['event']=='conversion'),['sku','condition']]

In [ ]:
df1['ventas'] = 1
grupo_ventas_celular = df1.groupby('condition').agg({'ventas':'sum'}) 

In [ ]:
g = grupo_ventas_celular['ventas'].sort_values(ascending= False).plot(kind= 'bar')
g.set_title('Cantidad de compras por condición del equipo')
g.set_xlabel('Condicion')
g.set_ylabel('Cantidad de compras')
g.set_xticklabels(['Bueno', 'Muy bueno', 'Excelente', 'Bueno - Sin touch ID', 'Nuevo'])

In [ ]:
datos_celulares = pd.merge(grupo_ventas_celular,cantidad_de_celulares,on='condition', how='left')

In [ ]:
datos_celulares

In [ ]:
datos_celulares['porcentaje vendido'] = datos_celulares['ventas']/datos_celulares['celulares']
datos_celulares

In [ ]:
ax = datos_celulares['porcentaje vendido'].plot(kind = 'bar')
ax.set_title('Porcentaje de ventas segun la condicion del celular')
ax.set_ylabel('Vendidos/Cantidad de celulares')
ax.set_xlabel('condicion')
plt.xticks(rotation=0)

In [ ]:
ax = datos_celulares[['ventas','celulares']].plot(kind = 'bar')
ax.set_title('ventas y cantidad de celulares')
ax.set_ylabel(' cantidad')
ax.set_xlabel('condicion')
plt.xticks(rotation=0)

Con las dos vistas, se puede observar, que la que más ventas tiene son los celulares que están en peor estado. Los celulares nuevos tienen alto grado de ventas, pero al ser infima la cantidad de celulares nuevos, no se puede asegurar nada.

In [ ]:
compras_por_almacenamiento = df.loc[(df['event']=='conversion'),['sku','storage']]
compras_por_almacenamiento['venta'] = 1
compras_por_almacenamiento.head()

In [ ]:
g = compras_por_almacenamiento.groupby('storage').agg(sum).sort_values(by='venta', ascending=False).plot(kind= 'bar', legend = False)
g.set_title('Cantidad de compras segun el espacio de almacenamiento del equipo')
g.set_xlabel('Espacio de almacenamiento')
g.set_ylabel('Cantidad de compras')


### Analisis duracion de sesion

In [ ]:
df_indice_en_persona_evento = df.set_index(['person','event'])

In [ ]:
dif_primer_ultimo_evento = df_indice_en_persona_evento.groupby('person').agg({'timestamp':['max', 'min']})
dif_primer_ultimo_evento['diferencia'] = dif_primer_ultimo_evento['timestamp']['max'] - dif_primer_ultimo_evento['timestamp']['min']
dif_primer_ultimo_evento.drop(columns='timestamp', inplace= True)
#mejora notoria de tiempo de ejecucion

In [ ]:
dif_primer_ultimo_evento = dif_primer_ultimo_evento.loc[dif_primer_ultimo_evento['diferencia']>'1 day'].sort_values(by = 'diferencia')
pd.concat([dif_primer_ultimo_evento.head(), dif_primer_ultimo_evento.tail()])

In [ ]:
eventos_por_persona = df.loc[(df['event'] != 'ad campaign hit') & (df['event'] != 'search engine hit'), ['person', 'timestamp', 'event']].copy()
eventos_por_persona['nueva_sesion'] = eventos_por_persona['event'] == 'visited site'
eventos_por_persona['timestamp_normalizado'] = eventos_por_persona['timestamp'].dt.round(freq = '20S') #agrupa de a ciclos de 10 segundos

#dejamos solo un evento por persona para cada ciclo de 10 segundos, dejando siempre los visited site o el evento que tiene un timestamp mas posterior
#eventos_por_persona = eventos_por_persona.sort_values(by = ['nueva_sesion', 'timestamp'], ascending = [False, False]).drop_duplicates(subset = ['person', 'timestamp_normalizado'], keep = 'first')
eventos_por_persona = eventos_por_persona.sort_values(['person', 'timestamp'])

#borrar eventos aislados que no sean visited site
eventos_por_persona['sig_timestamp'] = eventos_por_persona['timestamp'].shift(-1)
eventos_por_persona['ant_timestamp'] = eventos_por_persona['timestamp'].shift(1)
eventos_por_persona['dist_timestamps_ant'] = abs(eventos_por_persona['timestamp'] - eventos_por_persona['ant_timestamp'])
eventos_por_persona['dist_timestamps_sig'] = abs(eventos_por_persona['timestamp'] - eventos_por_persona['sig_timestamp'])
eventos_por_persona = eventos_por_persona.loc[((eventos_por_persona['dist_timestamps_ant'] < pd.Timedelta('1 hours')) \
                                               & (eventos_por_persona['dist_timestamps_sig'] < pd.Timedelta('1 hours')) \
                                               | eventos_por_persona['nueva_sesion'])]


eventos_por_persona['sig_nueva_sesion'] = eventos_por_persona['nueva_sesion'].shift(-1) #nueva columna que indica el valor 'nueva_sesion' de la siguiente fila
eventos_por_persona = eventos_por_persona.loc[(eventos_por_persona['nueva_sesion'] | eventos_por_persona['sig_nueva_sesion'])]
eventos_por_persona['sig_timestamp'] = eventos_por_persona['timestamp'].shift(-1)
eventos_por_persona = eventos_por_persona.loc[eventos_por_persona['nueva_sesion']]

eventos_por_persona.loc[(eventos_por_persona['nueva_sesion'] & eventos_por_persona['sig_nueva_sesion']), 'sig_timestamp'] = eventos_por_persona.loc[(eventos_por_persona['nueva_sesion'] & eventos_por_persona['sig_nueva_sesion']), 'timestamp']

eventos_por_persona['duracion_sesion'] = eventos_por_persona['sig_timestamp'] - eventos_por_persona['timestamp']
eventos_por_persona = eventos_por_persona.loc[((eventos_por_persona['duracion_sesion'] > pd.Timedelta('0 days')) & (eventos_por_persona['duracion_sesion'] < pd.Timedelta('6 hours')))]

sesiones_minutos = pd.Series()
sesiones_minutos = eventos_por_persona['duracion_sesion'] / pd.Timedelta(minutes=1)
#sesiones_minutos
ax = sesiones_minutos.hist(bins = range(0, 120, 1))
ax.set_yscale('log')
ax.set_xlabel('Duracion sesion en minutos')
ax.set_ylabel('Cantidad de sesiones')
ax.set_title('Distribución de duración de sesiones')

### Evolución de las campañas a lo largo del tiempo

Tomaremos las 5 campañas mas importantes y graficaremos su participiación durante el periodo.

In [ ]:
n = 5

hits_campanias = df.loc[(df['event']=='ad campaign hit'), ['timestamp', 'person', 'url', 'campaign_source']].copy()
hits_campanias['semana'] = hits_campanias['timestamp'].dt.week - 1
hits_campanias['semana'] = pd.to_datetime('2018', format='%Y') + pd.to_timedelta(hits_campanias['semana'].mul(7).astype('str') + ' days')
hits_campanias.dropna(subset = ['campaign_source'], inplace = True)

n_mayores_sources = hits_campanias['campaign_source'].value_counts().iloc[0:n]

hits_campanias = pd.crosstab(hits_campanias['semana'], hits_campanias['campaign_source'])
hits_campanias = hits_campanias.loc[:, n_mayores_sources.index]
# Porcentaje hits_campanias = hits_campanias.div(hits_campanias.sum(axis=1), axis=0).multiply(100)

ax = hits_campanias.plot(kind = 'area')
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
ax.set(xlabel='Mes', ylabel='Hits', title = 'Evolución de las '+str(n)+' mayores campañas publicitarias')

## ¿Qué campañas tuvieron mayor y menor alcance?

Medimos el alcance de una campaña como la cantidad de usuarios únicos que dirigieron a la página.

In [ ]:
ad_campaigns = df.loc[(df['event']=='ad campaign hit'),['person','campaign_source']].drop_duplicates()

### Mayor alcance

In [ ]:
N = 8
ad_campaigns_gruped = ad_campaigns.groupby('campaign_source').agg({'person': 'count'})
g = ad_campaigns_gruped.sort_values(by="person",ascending=False).head(N).iloc[::-1]\
    .plot(kind="barh",figsize=(9,3),legend=False)
g.set_title(str(N)+' campañas que mas personas dirigieron')
g.set_ylabel('Campaña publicitaria')
g.set_xlabel('Cantidad de personas')

### Menor alcance

In [ ]:
N = 8
g = ad_campaigns_gruped.sort_values(by="person",ascending=True).head(N)\
    .plot(kind="barh",figsize=(9,3),legend=False)
g.set_title(str(N)+' campañas que menos personas dirigieron')
g.set_ylabel('Campaña publicitaria')
g.set_xlabel('Cantidad de personas')

## ¿En qué cantidad de compras influyeron?


In [ ]:
personas_que_compraron = df.loc[df['event']=='conversion',['person']].drop_duplicates()
personas_que_compraron["compro"] = True

ad_compras = ad_campaigns.set_index("person")\
    .join(personas_que_compraron.set_index("person"),on="person",how="left")
ad_compras["person"] = ad_compras.index
ad_compras = ad_compras.groupby(["campaign_source"]).agg({"person":"count","compro":"count"})
ad_compras['porcentaje'] = 100* (ad_compras['compro']/ad_compras['person'])
ad_compras = ad_compras.loc[ad_compras["person"] >= 10,["porcentaje"]].sort_values(by="porcentaje",ascending=False)
p = ad_compras.iloc[::-1].plot(kind="barh",legend=False,figsize=(8,4),title="Porcentaje de compras por campaña publicitaria")
p.set_xlabel("Porcentaje")
p.set_ylabel("Campaña")

### Términos más buscados

Para analizar los términos, primero juntamos todas las palabras utilizadas en las búsquedas. Para evitar palabras duplicadas, las convertimos todas a mayúsculas.

In [ ]:
search_terms = df.loc[df["event"]== "searched products",["search_term"]]["search_term"].dropna().tolist()
text = " ".join(search_terms).upper()

La mejor forma de apreciar esta información es utilizando un gráfico donde el tamaño de cada palabra represente el número de apariciones en la búsqueda.
Con el fin de hacerlo mas agradable, utilizaremos el logo de Trocafone para darle forma al conjunto de palabras.

In [ ]:
# Carga de imagen
image_array = np.array(Image.open("imagenes/logo_500x500.png"))
# Conversion de pixeles para usarla como mascara
transformed_image_array = np.ndarray((image_array.shape[0],image_array.shape[1]), np.int32)
for i in range(len(image_array)):
    transformed_image_array[i] = list(map(lambda val: 0 if val == 0 else 255, image_array[i]))
    

In [ ]:
wc = WordCloud(max_words=2000,max_font_size=500,min_font_size=3\
               ,mask=transformed_image_array, background_color="white"\
               ,collocations=False # Considerar solo una palabra y no frases
              ).generate(text)

plt.figure(figsize=(8,8))
plt.imshow(wc,interpolation="kaiser")
plt.axis("off")
plt.show()

Se puede observar que la palabra mas buscada es 'IPHONE', seguido por SAMSUNG,GALAXY,6S,MOTO, entre otras.

### Analisis por dispositivo 

In [ ]:
df['device_type'].value_counts()

In [ ]:
dispositivos_registrados = df.loc[(df['event']=='visited site'),['person','device_type','new_vs_returning']]
dispositivos_nuevos = dispositivos_registrados.loc[dispositivos_registrados['new_vs_returning']=='New']

dispositivos_nuevos['device_type'].value_counts(normalize = True)

Vemos que aquellos dispositivos no identificados (Unknown) responde a una fraccion muy pequeña de los registros totales. Por lo tanto los desestimaremos en el análisis

In [ ]:
dispositivos_nuevos = dispositivos_nuevos.drop((dispositivos_nuevos[dispositivos_nuevos['device_type']=='Unknown']).index)

In [ ]:
g_dispositivos_nuevos = dispositivos_nuevos['device_type'].value_counts().iloc[:-1].plot('bar')
g_dispositivos_nuevos.set_title('Dispositivos registrados en nuevas visitas [New Hits]')
g_dispositivos_nuevos.set_xlabel('Tipo de dispositivo')
g_dispositivos_nuevos.set_ylabel('Cantidad de personas')

Podemos visualizar que los unicos dispositivos reelevantes sobre las nuevas visitas al  sitio son los Smartphones y las Computadoras

## Análisis sobre clientes que ingresaron por primera vez con Smartphone 

In [ ]:
#Filtro las personas entraron por primera vez por su celular
personas_nuevas_smartphone = dispositivos_nuevos.loc[(dispositivos_nuevos['device_type']=='Smartphone'),['person']]

#Filtro paralelamente las personas que volvieron a entrar desde sus PCs
personas_returning_pc = dispositivos_registrados.loc[(dispositivos_registrados['device_type']=='Computer') & (df['new_vs_returning']=='Returning'),['person']]
personas_returning_pc['Volvieron_en_PC'] = True

In [ ]:
personas_new_Smartphone_ret_PC = pd.merge(personas_nuevas_smartphone, personas_returning_pc, on='person', how='left').drop_duplicates()
personas_new_Smartphone_ret_PC['Volvieron_en_PC'].fillna(False,inplace=True)
personas_new_Smartphone_ret_PC.describe()

#### Veo la cantidad de personas que entraron por primera vez a la página desde sus celulares y volvieron desde sus PCs

In [ ]:
cantidad_personas_que_volvieron_enPC = personas_new_Smartphone_ret_PC['Volvieron_en_PC'].value_counts()
cantidad_personas_que_volvieron_enPC


In [ ]:
g_personas_ret_PC = cantidad_personas_que_volvieron_enPC.plot('bar')
g_personas_ret_PC.set_title('Personas que vuelven a la pagina desde la PC luego de entrar por primera vez desde Smartphone')
g_personas_ret_PC.set_xlabel('Vuelve a la pagina')
g_personas_ret_PC.set_ylabel('Cantidad de personas')
g_personas_ret_PC.set_xticklabels(['No','Sí'])


Se nota que no es representativa el numero de personas que entran primero a la pagina por sus celulares y luego vuelven a entrar desde la PC

#### Analizamos si de las personas que vuelven a entrar desde la PC (y que primero ingresaron con sus Smartphones) terminan realizando alguna compra

In [ ]:
personas_que_compraron = df.loc[df['event']=='conversion',['person']].drop_duplicates()
personas_que_compraron['compro'] = True
personas_que_compraron.head()

In [ ]:
personas_vuelven_y_compran = pd.merge(personas_new_Smartphone_ret_PC, personas_que_compraron, on='person', how='left').drop_duplicates()
personas_vuelven_y_compran['compro'].fillna(False,inplace=True)
cantidad_personas_que_vuelven_y_compran = personas_vuelven_y_compran.loc[personas_vuelven_y_compran['Volvieron_en_PC']==True]\
                ['compro'].value_counts()
cantidad_personas_que_vuelven_y_compran

In [ ]:
g5 = cantidad_personas_que_vuelven_y_compran.plot('bar',)
g5.set_title("Compradores que entraron por primera vez en Smartphone, y luego volvieron en PC ")
g5.set_xlabel("Compraron")
g5.set_ylabel("Cantidad de personas")
g5.set_xticklabels(['No','Sí'])
mostrar_porcentaje_barplot(g5)

Podemos ver que de las personas que volvieron a la pagina desde sus PCs luego de entrar desde sus SmartPhones, un 28,4% realizo compras

## Análisis sobre clientes que ingresaron por primera vez con PC 

In [ ]:
#Filtro las personas entraron por primera vez por su celular
personas_nuevas_pc = dispositivos_nuevos.loc[(dispositivos_nuevos['device_type']=='Computer'),['person']]

#Filtro paralelamente las personas que volvieron a entrar desde sus Smartphones
personas_returning_smartphone = dispositivos_registrados.loc[(dispositivos_registrados['device_type']=='Smartphone') & (df['new_vs_returning']=='Returning'),['person']]
personas_returning_smartphone['Volvieron_en_Smartphone'] = True

In [ ]:
personas_new_PC_ret_Smartphone = pd.merge(personas_nuevas_pc, personas_returning_smartphone, on='person', how='left').drop_duplicates()
personas_new_PC_ret_Smartphone['Volvieron_en_Smartphone'].fillna(False,inplace=True)
cantidad_personas_que_volvieron_Smartphone = personas_new_PC_ret_Smartphone['Volvieron_en_Smartphone'].value_counts()
cantidad_personas_que_volvieron_Smartphone

In [ ]:
g_personas_ret_Smartphones = cantidad_personas_que_volvieron_Smartphone.plot('bar')
g_personas_ret_Smartphones.set_title('Personas que vuelven a la pagina desde Smartphone luego de entrar por primera vez desde PC')
g_personas_ret_Smartphones.set_xlabel('Vuelve a la pagina')
g_personas_ret_Smartphones.set_ylabel('Cantidad de personas')
g_personas_ret_Smartphones.set_xticklabels(['No','Sí'])


In [ ]:
personas_vuelven_y_compran2 = pd.merge(personas_new_PC_ret_Smartphone, personas_que_compraron, on='person', how='left').drop_duplicates()
personas_vuelven_y_compran2['compro'].fillna(False,inplace=True)
cantidad_personas_vuelven_y_compran2 = personas_vuelven_y_compran2.loc[personas_vuelven_y_compran2['Volvieron_en_Smartphone']==True]['compro'].value_counts()

In [ ]:
g6 = cantidad_personas_vuelven_y_compran2.plot('bar',)
g6.set_title("Compradores que entraron por primera vez en PC, y luego volvieron en Smartphone ")
g6.set_xlabel("Compraron")
g6.set_ylabel("Cantidad de personas")
g6.set_xticklabels(['No','Sí'])
mostrar_porcentaje_barplot(g6)


### Para dar un punto de vista comparativo unificamos los plots anteriores

In [ ]:
concatenados_returning = pd.concat([cantidad_personas_que_volvieron_enPC, cantidad_personas_que_volvieron_Smartphone],axis=1)
concatenados_returning.columns = [' Migraron de Smartphone a PC','Migraron de PC a Smartphone']
g6 = concatenados_returning.T.plot(kind='barh')
g6.set_title('Usuarios que migraron desde su dispositivo inicial a otro')
g6.set_xlabel('Cantidad de personas')
g6.set_ylabel('Migracion')
g6.legend(['No', 'Sí'])

In [ ]:
concatenados_compra = pd.concat([cantidad_personas_que_vuelven_y_compran, cantidad_personas_vuelven_y_compran2], axis=1, )
concatenados_compra.columns = ['Smartphone a PC','PC a Smartphone']
g7 = concatenados_compra.T.plot(kind='barh')
g7.set_title('Compradores que migraron de un dispositivo a otro')
g7.set_xlabel('Cantidad de personas')
g7.set_ylabel('Migracion')
g7.legend(['No compro', 'Compro'])


#### Con pie chart para comparacion

In [ ]:
g = concatenados_returning.plot(kind='pie', subplots=True,labels= ['No', 'Si'],autopct='%1.1f%%' ,figsize=(12, 6),\
                               title='Usuarios que migraron desde su dispositivo inicial a otro')

In [ ]:
concatenados_compra.plot(kind='pie', subplots=True,labels= ['No compraron', 'Compraron'],autopct='%1.1f%%' ,figsize=(12, 6),\
                        title = 'Compradores que migraron de un dispositivo a otro')